# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here

orders = pd.read_csv('Orders.zip')
orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here

sub_prob1 = pd.DataFrame(orders.groupby(by='CustomerID')['amount_spent'].sum())
sub_prob1.sort_values(by = 'amount_spent',ascending=False)






quantile_95 = int(sub_prob1.quantile(q=0.95))
quantile_75 = int(sub_prob1.quantile(q=0.75))





sub_prob1.query('amount_spent >= @quantile_95')

sub_prob1.query('amount_spent < @quantile_95 and amount_spent > @quantile_75')





def customer_selection(x):
    if x >= quantile_95:
        return 'VIP'
    elif x > quantile_75 and x < quantile_95:
        return 'Prefered'
    else :
        return 'Thank you! Come again!'

sub_prob1['Customer_type'] = sub_prob1['amount_spent'].apply(customer_selection)
sub_prob1.rename(columns={'amount_spent':'Total_spent'},inplace=True)
sub_prob1

,Total_spent,Customer_type
CustomerID,,
12346,77183.60,VIP
12347,4310.00,Prefered
12348,1797.24,Prefered
12349,1757.55,Prefered
12350,334.40,Thank you! Come again!
...,...,...
18280,180.60,Thank you! Come again!
18281,80.82,Thank you! Come again!
18282,178.05,Thank you! Come again!


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [54]:
# your code here

orders_types = orders.merge(sub_prob1, on ='CustomerID', how = 'inner')

#pd.DataFrame(orders_types.groupby(['Country','Customer_type'])['amount_spent'].sum())
types = pd.get_dummies(orders_types['Customer_type'])
types['ind'] = types.index

orders_final = orders.merge(types, left_on = 'Unnamed: 0', right_on='ind', how='inner')
orders_final.drop(columns=['ind'],inplace=True)



q2 = orders_final.groupby(
   (['Country'])
 ).agg(
     Prefered =      ('Prefered','sum'),
     Thank_you =     ('Thank you! Come again!','sum'),
     VIP=            ('VIP','sum')
 )

q2.sort_values(by='VIP',ascending=False)

,Prefered,Thank_you,VIP
Country,,,
United Kingdom,96787.0,92991.0,65125.0
Germany,2516.0,2844.0,1588.0
France,2449.0,2495.0,1319.0
EIRE,2281.0,2026.0,1099.0
Netherlands,734.0,545.0,533.0
Spain,839.0,596.0,397.0
Switzerland,565.0,655.0,272.0
Portugal,371.0,557.0,259.0
Norway,207.0,256.0,252.0


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [66]:
# your code here

data = pd.DataFrame(orders_final.groupby(
   (['Country'])
 ).agg(
     Thank_you =     ('Thank you! Come again!','sum'),
     VIP=            ('VIP','sum')
    
    
 ).sum(axis=1))



data.columns=['VIP+Prefered']

data.sort_values(by='VIP+Prefered',ascending=False)


,VIP+Prefered
Country,
United Kingdom,158116.0
Germany,4432.0
France,3814.0
EIRE,3125.0
Netherlands,1078.0
Spain,993.0
Belgium,973.0
Switzerland,927.0
Australia,868.0
